<!-- @format -->

# Zadanie 2

(2p)

Zmodyfikuj notatnik **add_demo.ipynb** (ten sam, co w poprzednim zadaniu), żeby wykonywał on mnożenie dwóch liczb trzycyfrowych, generując jakieś pośrednie kroki obliczeń (na przykład realizując mnożenie na piśmie). Postaraj się osiągnąć poziom skuteczności istotnie wyższy niż w poprzednim zadaniu.

Za zadanie jest premia równa $2*x$, gdzie $x$ jest punktacją na zbiorze treningowym


<!-- @format -->

A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.


In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed

set_seed(3407)

In [2]:
lst = [[1, 2, 3], [4, 5, 6]]

x = [n for nums in lst for n in nums]
x

[1, 2, 3, 4, 5, 6]

In [3]:
lst = [1, 2, 3] + [1, 2, 3]
lst

[1, 2, 3, 1, 2, 3]

In [2]:
import random


def random_add_instance(length):
    a = [random.randint(0, 9) for i in range(length)]
    b = [random.randint(0, 9) for i in range(length)]
    val_a = int("".join(str(d) for d in a))
    val_b = int("".join(str(d) for d in b))
    val_c = val_a + val_b
    str_c = str(val_c)
    str_c = (length + 1 - len(str_c)) * "0" + str_c
    return a + b + [int(d) for d in str_c]


def sub_mult_instance(a, b):
    xyz = [a * int(d) for d in str(b)]
    # if len(xyz) < 3:
    #     print(xyz, a, b)
    print(xyz, a, b)
    if len(xyz) < 3:
        for _ in range(3 - len(xyz)):
            xyz.append(0)

    res = []
    for val in xyz:
        str_val = str(val)
        str_val = (4 - len(str_val)) * "0" + str_val
        res.extend(int(d) for d in str_val)

    # return [(4 - len(sv := str(val))) * "0" + sv for val in xyz]

    # if len(xyz) < 3:
    #     for _ in range(3 - len(xyz)):
    #         res.extend([0, 0, 0, 0])

    return res


def random_mult_instance(length):
    a = [random.randint(0, 9) for i in range(length)]
    b = [random.randint(0, 9) for i in range(length)]
    val_a = int("".join(str(d) for d in a))
    val_b = int("".join(str(d) for d in b))
    val_c = val_a * val_b
    str_c = str(val_c)

    xyz = sub_mult_instance(val_a, val_b)

    str_c = (length * 2 - len(str_c)) * "0" + str_c
    return a + b + xyz + [int(d) for d in str_c]


for i in range(10):
    print(random_mult_instance(3))

sub_mult_instance(896, 999)

[310, 186, 372] 62 536
[0, 6, 2, 5, 3, 6, 0, 3, 1, 0, 0, 1, 8, 6, 0, 3, 7, 2, 0, 3, 3, 2, 3, 2]
[1485, 3465, 3960] 495 378
[4, 9, 5, 3, 7, 8, 1, 4, 8, 5, 3, 4, 6, 5, 3, 9, 6, 0, 1, 8, 7, 1, 1, 0]
[1932, 644, 2576] 644 314
[6, 4, 4, 3, 1, 4, 1, 9, 3, 2, 0, 6, 4, 4, 2, 5, 7, 6, 2, 0, 2, 2, 1, 6]
[359, 359, 3231] 359 119
[3, 5, 9, 1, 1, 9, 0, 3, 5, 9, 0, 3, 5, 9, 3, 2, 3, 1, 0, 4, 2, 7, 2, 1]
[350, 1400, 2800] 350 148
[3, 5, 0, 1, 4, 8, 0, 3, 5, 0, 1, 4, 0, 0, 2, 8, 0, 0, 0, 5, 1, 8, 0, 0]
[18, 6, 14] 2 937
[0, 0, 2, 9, 3, 7, 0, 0, 1, 8, 0, 0, 0, 6, 0, 0, 1, 4, 0, 0, 1, 8, 7, 4]
[1696, 2544, 5936] 848 237
[8, 4, 8, 2, 3, 7, 1, 6, 9, 6, 2, 5, 4, 4, 5, 9, 3, 6, 2, 0, 0, 9, 7, 6]
[6993, 0, 5994] 999 706
[9, 9, 9, 7, 0, 6, 6, 9, 9, 3, 0, 0, 0, 0, 5, 9, 9, 4, 7, 0, 5, 2, 9, 4]
[745, 2235, 4470] 745 136
[7, 4, 5, 1, 3, 6, 0, 7, 4, 5, 2, 2, 3, 5, 4, 4, 7, 0, 1, 0, 1, 3, 2, 0]
[404, 202] 101 42
[1, 0, 1, 0, 4, 2, 0, 4, 0, 4, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 4, 2, 4, 2]
[8064, 8064, 8064] 896 999


[8, 0, 6, 4, 8, 0, 6, 4, 8, 0, 6, 4]

In [5]:
l = [0, 1, 2, 3, 4, 5, 6, 7, 8]
print(l[:5])

[0, 1, 2, 3, 4]


In [6]:
class MultDataset(Dataset):
    """
    Dataset for the Add problem. E.g. for problem length 3:
    12 + 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 3 4
    output: I I I I I 0 3 4 5
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {"train", "test"}
        self.split = split
        self.length = length

    def __len__(self):
        return 10000  # ...

    def get_vocab_size(self):
        return 10

    def get_block_size(self):
        # the length of the sequence that will feed into transformer,
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element

        # a(3) * b(3) = a * b_0(1) + a * b_1(1) + a * b_2(1) = x(4) + y(4) + z(4) = c(6)
        # 3 + 3 + 3 + 4 + 4 + 4 + 6
        return 9 * self.length + 1 - 1

    def __getitem__(self, idx):
        while True:
            rai = random_mult_instance(self.length)
            h = hash(str(rai[: 2 * self.length]))

            inp_split = (
                "test" if h % 4 == 0 else "train"
            )  # designate 25% of examples as test
            if inp_split == self.split:
                break  # ok

        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)

        # we only want to predict at output locations, mask out the loss at the input locations
        y[: 6 * self.length - 1] = -1
        return x, y


# """
# in:  8 9 6 | 9 9 9 | 8 0 6 4 8 0 6 4 8 0 6 4 | 8 9 5 1 0
# out: I I I | I I I | I I I I I I I I I I I 8 | 9 5 1 0 4
# """

In [7]:
# print an example instance of the dataset
train_dataset = MultDataset("train")
test_dataset = MultDataset("test")
x, y = train_dataset[0]

print(x)
for a, b in zip(x, y):
    print(int(a), int(b))

tensor([2, 9, 3, 4, 8, 0, 1, 1, 7, 2, 2, 3, 4, 4, 0, 0, 0, 0, 1, 4, 0, 6, 4])
2 -1
9 -1
3 -1
4 -1
8 -1
0 -1
1 -1
1 -1
7 -1
2 -1
2 -1
3 -1
4 -1
4 -1
0 -1
0 -1
0 -1
0 1
1 4
4 0
0 6
6 4
4 0


In [8]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = "gpt-micro"
# model_config.model_type = 'gpt-nano'

model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.80M


In [9]:
print(model_config.n_head, model_config.n_layer, model_config.n_embd)

4 4 128


In [10]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
# train_config.learning_rate = (
#     2e-5  # the model we're using is so small that we can go a bit faster
# )
# train_config.lr_decay = True
train_cofig.learning_rate = 5e-4
train_config.max_iters = 25000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

running on device cuda


In [11]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(
            f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}"
        )


trainer.set_callback("on_batch_end", batch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 2.33865
iter_dt 17.24ms; iter 100: train loss 2.12334
iter_dt 22.26ms; iter 200: train loss 2.13650
iter_dt 18.12ms; iter 300: train loss 2.06674
iter_dt 18.12ms; iter 400: train loss 2.03811
iter_dt 19.00ms; iter 500: train loss 1.91560
iter_dt 19.11ms; iter 600: train loss 1.81487
iter_dt 23.30ms; iter 700: train loss 1.80654
iter_dt 22.26ms; iter 800: train loss 1.75082
iter_dt 23.22ms; iter 900: train loss 1.71938
iter_dt 20.21ms; iter 1000: train loss 1.66311
iter_dt 17.22ms; iter 1100: train loss 1.64150
iter_dt 17.16ms; iter 1200: train loss 1.54128
iter_dt 18.12ms; iter 1300: train loss 1.60352
iter_dt 17.20ms; iter 1400: train loss 1.51160
iter_dt 19.19ms; iter 1500: train loss 1.47088
iter_dt 23.30ms; iter 1600: train loss 1.52189
iter_dt 24.00ms; iter 1700: train loss 1.45035
iter_dt 20.28ms; iter 1800: train loss 1.40910
iter_dt 23.27ms; iter 1900: train loss 1.36974
iter_dt 20.17ms; iter 2000: train loss 1.36081
iter_dt 17.25ms; iter 2100

In [12]:
# now let's perform some evaluation
model.eval()
None

In [13]:
def eval_mult_split(trainer, split, max_batches):
    dataset = {"train": train_dataset, "test": test_dataset}[split]
    n = train_dataset.length  # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)

    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        """ 
            Dataset for the Add problem. E.g. for problem length 3:
            12 + 333 = 345
            Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
            Which will feed into the transformer concatenated as:
            input:  0 1 2 3 3 3 0 3 4
            output: I I I I I 0 3 4 5
            where I is "ignore", as the transformer is reading the input sequence
            
        """

        """
        in:  8 9 6 | 9 9 9 | 8 0 6 4 8 0 6 4 8 0 6 4 | 8 9 5 1 0
        out: I I I | I I I | I I I I I I I I I I I 8 | 9 5 1 0 4
         """

        inp = x[:, : 6 * n]
        sol = y[:, -n * 2 :]

        cat = model.generate(
            inp, n * 2, do_sample=False
        )  # using greedy argmax, not sampling
        sol_candidate = cat[:, -n * 2 :]
        correct = (sol == sol_candidate).all(1).cpu()
        for i in range(x.size(0)):
            results.append(int(correct[i]))

    rt = torch.tensor(results, dtype=torch.float)
    print(
        "%s final score: %d/%d = %.2f%% correct"
        % (split, rt.sum(), len(results), 100 * rt.mean())
    )
    return rt.sum()


# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_mult_split(trainer, "train", max_batches=50)
    test_score = eval_mult_split(trainer, "test", max_batches=50)

train final score: 9199/10000 = 91.99% correct
test final score: 9190/10000 = 91.90% correct


In [14]:
# Where we made changes to the code
# 1. We changed instance function
# 2. In rai = random_add_instance(self.length) we changed add to mult
# 3. We incresed get_block_size to 4 * self.length + 1 - 1
# 4. In eval_mult_split we changed the inp, sol and sol_candidate size from n+1 to n * 2
# 5. In cat = model.generate(inp, n * 2, do_sample=False) we changed n+1 to n * 2
# 6. We changed learning rate to 2e-6